# Setup

In [ ]:
!pip install tensorflow.gpu

     |████████████████████████████████| 394.3MB 43kB/s 


## Libraries




In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model

from tensorflow.keras.applications.vgg16 import VGG16 #change import to get different architectures

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential

import numpy as np

from glob import glob

# Dataset

In [ ]:
# Image resizing for VGG16 used here
IMAGE_SIZE = [224, 224]

# Pathway, all files are saved in google drive
test_train_ratio = '70_30'  # User input

base_dir = '/content/drive/MyDrive/Datasets/'

train_path = base_dir + test_train_ratio + '/train'
valid_path = base_dir + test_train_ratio + '/valid' # Testing path**

In [ ]:
# Get the number of output classes
# folders = glob('/content/drive/MyDrive/Datasets/80_20/train/*')
folders = glob(train_path + '/*')

folders #get all the folders

['/content/drive/MyDrive/Datasets/70_30/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 '/content/drive/MyDrive/Datasets/70_30/train/Tomato___healthy',
 '/content/drive/MyDrive/Datasets/70_30/train/Tomato___Septoria_leaf_spot',
 '/content/drive/MyDrive/Datasets/70_30/train/Tomato___Late_blight',
 '/content/drive/MyDrive/Datasets/70_30/train/Tomato___Spider_mites Two-spotted_spider_mite',
 '/content/drive/MyDrive/Datasets/70_30/train/Tomato___Target_Spot',
 '/content/drive/MyDrive/Datasets/70_30/train/Tomato___Tomato_mosaic_virus',
 '/content/drive/MyDrive/Datasets/70_30/train/Tomato___Leaf_Mold',
 '/content/drive/MyDrive/Datasets/70_30/train/Tomato___Bacterial_spot',
 '/content/drive/MyDrive/Datasets/70_30/train/Tomato___Early_blight']

## Data Augmentation

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All are rescaled (colour)
train_datagen = ImageDataGenerator(rescale = 1./255),
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

## Data Feeding


In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path ,
target_size = (224, 224),
batch_size = 16,
class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(valid_path,
target_size = (224, 224),
batch_size = 16,
class_mode = 'categorical')

Found 16052 images belonging to 10 classes.
Found 6878 images belonging to 10 classes.


# Model

## Model Setup

In [ ]:
# Weights is initialised using imagenet

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 3s 0us/step


In [ ]:
# don't train existing weights
for layer in vgg16.layers:
	layer.trainable = False


In [ ]:
x = Flatten()(vgg16.output)

# softmax
prediction = Dense(len(folders), activation= 'softmax')(x)

# create a model
model = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
model.compile(
	loss='categorical_crossentropy',
	optimizer='adam',
	metrics=['accuracy']
)

## Train & Eval 1


In [ ]:
# Loads the weights
base_dir_model = '/content/drive/MyDrive/Models/No Aug/'
arch = 'VGG16/' # User input (make sure there's a '/')
current_ep = 2  # User input, starts from 0

if current_ep > 0:
  load_ep = '/ep' + str(current_ep) + '/ep' + str(current_ep) +'.ckpt' 
  load_path = base_dir_model + arch + test_train_ratio + load_ep
  model.load_weights(load_path)

In [ ]:
num_epochs = 1  # User input, number of epochs to train

# save weights during training
checkpoint_ep = '/ep' + str(current_ep+1) + '/ep' + str(current_ep+1) +'.ckpt' 
checkpoint_path = base_dir_model + arch + test_train_ratio + checkpoint_ep

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(training_set,
          epochs=num_epochs,
          callbacks=[cp_callback])  # Pass callback to training

In [ ]:
# Model Evaluation
print("Evaluate on test data")
results = model.evaluate(test_set, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
430/430 [==============================] - 3438s 8s/step - loss: 0.6086 - accuracy: 0.8672
test loss, test acc: [0.5773594379425049, 0.8692933917045593]


## Train & Eval 2

In [ ]:
# Loads the weights
current_ep = 4  # starts from 0

if current_ep > 0:
  load_ep = '/ep' + str(current_ep) + '/ep' + str(current_ep) +'.ckpt' 
  load_path = base_dir_model + arch + test_train_ratio + load_ep
  model.load_weights(load_path)

In [ ]:
# num_epochs = 1  # User input, number of epochs to train

# # save weights during training
# checkpoint_ep = '/ep' + str(current_ep+1) + '/ep' + str(current_ep+1) +'.ckpt' 
# checkpoint_path = base_dir_model + arch + test_train_ratio + checkpoint_ep

# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

# # Train the model with the new callback
# model.fit(training_set,
#           epochs=num_epochs,
#           callbacks=[cp_callback])  # Pass callback to training

In [ ]:
# Model Evaluation
print("Evaluate on test data")
results = model.evaluate(test_set, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
430/430 [==============================] - 3517s 8s/step - loss: 0.5283 - accuracy: 0.8689
test loss, test acc: [0.528264582157135, 0.868857204914093]


## Train & Eval 3

In [ ]:
# Loads the weights
current_ep += 1  # starts from 0

if current_ep > 0:
  load_ep = '/ep' + str(current_ep) + '/ep' + str(current_ep) +'.ckpt' 
  load_path = base_dir_model + arch + test_train_ratio + load_ep
  model.load_weights(load_path)

In [ ]:
# num_epochs = 1  # User input, number of epochs to train

# # save weights during training
# checkpoint_ep = '/ep' + str(current_ep+1) + '/ep' + str(current_ep+1) +'.ckpt' 
# checkpoint_path = base_dir_model + arch + test_train_ratio + checkpoint_ep

# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

# # Train the model with the new callback
# model.fit(training_set,
#           epochs=num_epochs,
#           callbacks=[cp_callback])  # Pass callback to training

In [ ]:
# Model Evaluation
print("Evaluate on test data")
results = model.evaluate(test_set, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
430/430 [==============================] - 3560s 8s/step - loss: 0.5283 - accuracy: 0.8689
test loss, test acc: [0.5282646417617798, 0.868857204914093]


## Train & Eval 4

In [ ]:
# Loads the weights
current_ep = current_ep + 1  # starts from 0

if current_ep > 0:
  load_ep = '/ep' + str(current_ep) + '/ep' + str(current_ep) +'.ckpt' 
  load_path = base_dir_model + arch + test_train_ratio + load_ep
  model.load_weights(load_path)

In [ ]:
# num_epochs = 1  # User input, number of epochs to train

# # save weights during training
# checkpoint_ep = '/ep' + str(current_ep+1) + '/ep' + str(current_ep+1) +'.ckpt' 
# checkpoint_path = base_dir_model + arch + test_train_ratio + checkpoint_ep

# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

# # Train the model with the new callback
# model.fit(training_set,
#           epochs=num_epochs,
#           callbacks=[cp_callback])  # Pass callback to training

In [ ]:
# Model Evaluation
print("Evaluate on test data")
results = model.evaluate(test_set, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
430/430 [==============================] - 3390s 8s/step - loss: 0.5774 - accuracy: 0.8693
test loss, test acc: [0.5773590207099915, 0.8692933917045593]


## Train & Eval 5

In [ ]:
# # Loads the weights
# current_ep = current_ep + 1  # starts from 0

# if current_ep > 0:
#   load_ep = '/ep' + str(current_ep) + '/ep' + str(current_ep) +'.ckpt' 
#   load_path = base_dir_model + arch + test_train_ratio + load_ep
#   model.load_weights(load_path)

In [ ]:
# num_epochs = 1  # User input, number of epochs to train

# # save weights during training
# checkpoint_ep = '/ep' + str(current_ep+1) + '/ep' + str(current_ep+1) +'.ckpt' 
# checkpoint_path = base_dir_model + arch + test_train_ratio + checkpoint_ep

# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

# # Train the model with the new callback
# model.fit(training_set,
#           epochs=num_epochs,
#           callbacks=[cp_callback])  # Pass callback to training

In [ ]:
# # Model Evaluation
# print("Evaluate on test data")
# results = model.evaluate(test_set, batch_size=128)
# print("test loss, test acc:", results)

# Data Visualisation

Accuracy and Validation Accuracy

In [ ]:
# import matplotlib.pyplot as plt

# acc = hist.history['accuracy']
# # val_acc = hist.history['val_accuracy']

# epochs = range(1,num_epochs+1)
# plt.plot(epochs, acc, 'g', label='Training accuracy')
# plt.plot(epochs, val_acc, 'b', label='Validation accuracy')

# #Labels and plot
# plt.title('Training and Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

Loss and Validation Loss

In [ ]:
# loss = hist.history['loss']
# val_loss = hist.history['val_loss']

# epochs = range(1,num_epochs+1)
# plt.plot(epochs, loss, 'r', label='Loss')
# plt.plot(epochs, val_loss, 'c', label='Validation Loss')

# #Labels and plot
# plt.title('Loss and Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()